In [1]:
# --- Core Libraries ---
import os
import random
import json
import pandas as pd
import numpy as np
import torch
import time
from tqdm import tqdm
import re

# --- Hugging Face: Dataset, Tokenizer, Model ---
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    Trainer, 
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    pipeline
)

# --- LoRA & Parameter-Efficient Tuning ---
from peft import LoraConfig, get_peft_model, TaskType

# --- W&B Experiment Tracking ---
import wandb

# --- SQL Evaluation ---
import sqlite3
import sqlparse
from tabulate import tabulate
import evaluate  # for BLEU, ROUGE
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sidpk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sidpk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
os.environ["WANDB_NOTEBOOK_NAME"] = "text2sql_finetune_and_eval.ipynb"

In [3]:
torch.cuda.empty_cache()

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    print("GPU not detected — will fall back to CPU.")

PyTorch version: 2.5.1+cu121
CUDA available: True
Using GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [4]:
# Load dataset
dataset = load_dataset("Clinton/Text-to-SQL-v1")
shuffled_dataset = dataset.shuffle(seed=42)

df = pd.DataFrame(shuffled_dataset["train"])
df.sample(5)

,instruction,input,response,source,text
172776,What rowers have a 6:29.56 time?,"CREATE TABLE table_62947 (\n ""Rank"" real,\n...","SELECT ""Rowers"" FROM table_62947 WHERE ""Time"" ...",wikisql,Below are sql tables schemas paired with instr...
55857,What is the average capacity for the vehicle h...,"CREATE TABLE table_65050 (\n ""Driver"" text,...","SELECT AVG(""Capacity"") FROM table_65050 WHERE ...",wikisql,Below are sql tables schemas paired with instr...
107337,Find the names of nurses who are on call.,"CREATE TABLE medication (\n code number,\n ...",SELECT DISTINCT T1.name FROM nurse AS T1 JOIN ...,spider,Below are sql tables schemas paired with instr...
230584,What title was used in the nomination of the S...,"CREATE TABLE table_65600 (\n ""Year (Ceremon...","SELECT ""Film title used in nomination"" FROM ta...",wikisql,Below are sql tables schemas paired with instr...
171681,List the types of competition and the number o...,"CREATE TABLE club (\n Club_ID int,\n nam...","SELECT Competition_type, COUNT(*) FROM competi...",nvbench,Below are sql tables schemas paired with instr...


In [5]:
print("Any nulls?", df.isna().sum())
print("Any empty strings?", (df == "").sum())
print("Unique columns:", df.columns)

Any nulls? instruction    0
input          0
response       0
source         0
text           0
dtype: int64
Any empty strings? instruction    2
input          0
response       0
source         0
text           0
dtype: int64
Unique columns: Index(['instruction', 'input', 'response', 'source', 'text'], dtype='object')


In [6]:
df_clean = df[df["instruction"] != ""].reset_index(drop=True)
print(f"Filtered dataset size: {len(df_clean)}")

Filtered dataset size: 262206


In [7]:
formatted_dataset = Dataset.from_pandas(df_clean[["text"]])
formatted_dataset = formatted_dataset.train_test_split(test_size=0.1, seed=42)

print(formatted_dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 235985
    })
    test: Dataset({
        features: ['text'],
        num_rows: 26221
    })
})


In [8]:
# Load Tokenizer

model_name = "deepseek-ai/deepseek-coder-1.3b-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [9]:
# Find max length of instructions to pick the optimal max prompt length

# Function to compute token length stats
def compute_token_stats(dataset_split, tokenizer):
    lengths = [len(tokenizer(x)["input_ids"]) for x in dataset_split["text"]]
    stats = {
        "max": int(np.max(lengths)),
        "95th_percentile": int(np.percentile(lengths, 95)),
        "mean": round(np.mean(lengths), 2),
        "min": int(np.min(lengths)),
        "num_samples": len(lengths),
    }
    return stats

# Compute for both splits
train_stats = compute_token_stats(formatted_dataset["train"], tokenizer)
test_stats = compute_token_stats(formatted_dataset["test"], tokenizer)

print("Train Token Length Stats:", train_stats)
print("Test Token Length Stats:", test_stats)

Train Token Length Stats: {'max': 3226, '95th_percentile': 1435, 'mean': np.float64(377.15), 'min': 66, 'num_samples': 235985}
Test Token Length Stats: {'max': 3218, '95th_percentile': 1420, 'mean': np.float64(376.35), 'min': 69, 'num_samples': 26221}


In [10]:
#looking at the max token size in the entire data response
sql_token_lengths = df_clean["response"].apply(lambda x: len(tokenizer(x, truncation=False)["input_ids"]))

# Analyze
print("Mean SQL Response token length:", sql_token_lengths.mean())
print("95th percentile:", sql_token_lengths.quantile(0.95))
print("Max SQL Response token length:", sql_token_lengths.max())

Mean SQL Response token length: 51.61714834900803
95th percentile: 162.0
Max SQL Response token length: 1868


In [11]:
#Smart Padding
def tokenize(examples):
    input_ids_list = []
    attention_mask_list = []
    labels_list = []
    
    max_length = 4096

    for full_text in examples["text"]:
        # Extract prompt and response
        prompt_text = full_text.split("### Response:")[0].strip() + "\n### Response:\n"
        response_text = full_text.split("### Response:")[1].strip()
        
        # Tokenize with truncation
        prompt_tokens = tokenizer(prompt_text, truncation=True, max_length=max_length)["input_ids"]
        response_tokens = tokenizer(response_text, truncation=True, max_length=max_length)["input_ids"]
        response_tokens.append(tokenizer.eos_token_id)
        
        # Combine tokens for input
        input_ids = prompt_tokens + response_tokens
        attention_mask = [1] * len(input_ids)
        
        # Create labels - keep prompt tokens, mask response tokens
        labels = input_ids.copy()  # Start with full sequence
        labels = [-100] * len(prompt_tokens) + response_tokens #mask prompt tokens

        input_ids_list.append(input_ids)
        attention_mask_list.append(attention_mask)
        labels_list.append(labels)

    return {
        "input_ids": input_ids_list,
        "attention_mask": attention_mask_list,
        "labels": labels_list
    }

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # because this is causal LM
    pad_to_multiple_of=16  # speeds up training on GPU
)

In [12]:
#computing the metrics for the baseline model based on similarilty of output, sql compilation and time

# Load metrics
meteor_metric = evaluate.load("meteor")

def can_execute_sql(generated_sql, schema=None):
    """Check if a SQL query can be executed against a given schema.
    
    Args:
        generated_sql (str): The SQL query to test
        schema (str, optional): The database schema to create before testing
        
    Returns:
        bool: True if the query executes successfully, False otherwise
    """
    try:
        conn = sqlite3.connect(":memory:")
        cursor = conn.cursor()
        
        # Create schema if provided
        if schema:
            cursor.executescript(schema)
            
        # Try to execute the query
        cursor.execute(generated_sql)
        return True
    except sqlite3.Error as e:
        print(f"SQL Error: {e}")
        return False
    finally:
        conn.close()

def extract_sql_from_output(output_text, prompt_text):
    """Extract SQL query from model output, handling various formats."""
    # Remove the prompt from the output
    sql_text = output_text[len(prompt_text):].strip()
    
    # Remove any markdown code blocks if present
    sql_text = re.sub(r'```sql\s*|\s*```', '', sql_text)
    sql_text = re.sub(r'```\s*|\s*```', '', sql_text)
    
    # Remove any trailing text after semicolon
    if ';' in sql_text:
        sql_text = sql_text.split(';')[0] + ';'
    
    return sql_text.strip()

def evaluate_model_on_dataset(
    model,
    tokenizer,
    dataset,
    max_new_tokens=2048
):
    predictions = []
    references = []
    compile_success = 0
    execution_times = []

    dataset_slice = dataset

    for example in tqdm(dataset_slice, desc="Evaluating"):
        # Extract prompt and response using the same format as tokenize function
        prompt_text = example["text"].split("### Response:")[0].strip() + "\n### Response:\n"
        ground_truth = example["text"].split("### Response:")[1].strip()
        schema = example["text"].split("### Input:")[1].split("### Response:")[0].strip()

        inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                eos_token_id=tokenizer.eos_token_id,
                max_new_tokens=max_new_tokens,
                pad_token_id=tokenizer.eos_token_id
                )
        
        # Get the generated SQL - everything after the prompt
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_sql = extract_sql_from_output(decoded, prompt_text)
        print("SQL Output:", generated_sql)

        # Add prediction for METEOR
        predictions.append(generated_sql)
        references.append([ground_truth])  # METEOR expects references as a list of lists

        # Compile SQL Query and measure time
        start_time = time.perf_counter()
        success = can_execute_sql(generated_sql, schema)
        end_time = time.perf_counter()

        if success:
            compile_success += 1
            execution_times.append(end_time - start_time)

    # Compute metrics
    meteor_score = meteor_metric.compute(predictions=predictions, references=references)["meteor"]
    sql_compilation_rate = compile_success / len(dataset_slice)
    
    # Calculate average execution time for successful queries
    avg_execution_time = sum(execution_times) / len(execution_times) if execution_times else 0

    metrics = {
        "meteor_score": round(meteor_score, 4),
        "sql_compilation_rate": round(sql_compilation_rate, 4),
        "avg_execution_time_ms": round(avg_execution_time * 1000, 2),  # Convert to milliseconds
        "num_eval_samples": len(dataset_slice),
        "num_successful_queries": compile_success
    }

    return metrics


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sidpk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\sidpk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sidpk\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [13]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

In [14]:
#starting the finetuning process

In [15]:
torch.cuda.empty_cache()

In [16]:
wandb.init(
    project="deepseek-sql-finetune",
    name="baseline-run",
    notes="1.3B model with QLoRA, loss tracking"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sidpkul21 (sidpkul21-georgia-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [17]:
training_args = TrainingArguments(
    output_dir="./deepseek-coder-qlora-sql-v2",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    num_train_epochs=3,
    logging_steps=100,
    save_steps=500,
    fp16=True,
    report_to="wandb",
    run_name="deepseek-coder-qlora-sql-run1",
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=3,
    label_names=["labels"] 
)

In [18]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # or ["query_key_value"] depending on model architecture
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model_finetune = get_peft_model(model, lora_config)

In [19]:
tokenized_dataset = formatted_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/235985 [00:00<?, ? examples/s]

Map:   0%|          | 0/26221 [00:00<?, ? examples/s]

In [20]:
small_train = tokenized_dataset["train"].select(range(10000))
small_eval = tokenized_dataset["test"].select(range(1000))

trainer = Trainer(
    model=model_finetune,
    args=training_args,
    train_dataset=small_train, 
    eval_dataset=small_eval,
    tokenizer=tokenizer,
    data_collator=data_collator
)

C:\Users\sidpk\AppData\Local\Temp\ipykernel_3000\2103303720.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.844200,0.893414
1000,0.708400,0.758137
1500,0.636500,0.694039
2000,0.600400,0.672734
2500,0.563600,0.660649
3000,0.565100,0.655386
3500,0.544700,0.654213


TrainOutput(global_step=3750, training_loss=0.7051525970458984, metrics={'train_runtime': 11637.7342, 'train_samples_per_second': 2.578, 'train_steps_per_second': 0.322, 'total_flos': 8.956591972201267e+16, 'train_loss': 0.7051525970458984, 'epoch': 3.0})

In [22]:
model_finetune.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32256, 2048)
        (layers): ModuleList(
          (0-23): 24 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear

In [63]:
prompt = """	
Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: What model has a launch of September 3, 2010? ### Input: CREATE TABLE table_28269 (
"Model" text,
"Launch" text,
"Code name" text,
"Transistors (million)" real,
"Die size (mm 2 )" real,
"Bus interface" text,
"Memory ( MB )" text,
"SM count" real,
"Core config 1,3" text,
"Core ( MHz )" real,
"Shader ( MHz )" real,
"Memory ( MHz )" text,
"Pixel ( GP /s)" text,
"Texture ( GT /s)" text,
"Bandwidth ( GB /s)" text,
"DRAM type" text,
"Bus width ( bit )" real,
"GFLOPS (FMA) 2" text,
"TDP (watts)" real,
"Release price (USD)" text
) ### Response:
"""

In [65]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model_finetune.generate(
    **inputs,
    max_new_tokens=2048,
    temperature=0.2,
    top_p=0.95,
    do_sample=True,
    eos_token_id = tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.2,  # Add repetition penalty
    no_repeat_ngram_size=3,  # Prevent repeating 3-grams
)

generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
# Extract only the response part (everything after "### Response:")
generated_sql = generated_sql.split("### Response:")[-1].strip()
print(generated_sql)

SELECT "model", COUNT("launch") FROM table_547 WHERE DATETIME(PARSE('September  3 ,   2001' USING 'MMM dd'), PARSE('January    1,' USING '%Y')) = LAUNCH GROUP BY MODEL ORDER BY DESC LIMIT


In [25]:
# MISSES FROM SECOND TRAINING
#The training data was not large enough to train the model to its full potential.
